# Тестовое задание от VK

## Импортируем библиотеки
> Пригодятся только для тестирования времени выполнения программы. В самой программе они не требуются.

In [1]:
import lorem # https://pypi.org/project/python-lorem/
import random

## Вводим используемые в проекте функции

### rounder_upper - окруление числа в большую сторону

На вход поступает `float` число, из которого выходит `int`, округленное в большую сторону
```
3.45 -> 4
5.7 -> 6
9.0 -> 9
```

In [2]:
def rounder_upper(num: float) -> int:
    "Округление float числа в большую сторону"
    int_part = int(str(num).split('.')[0])
    if num == int_part:
        return int(num)
    else:
        return int(num + (1-(num - int_part)))

### calculate_suffix_len - расчет длины суффикса в зависимости от количества секций в выходной строке

На вход поступает длина исходной строки (с учетом пробелов) в `int`. На выходе получаем необходимое количество символов, которые нужно оставить "про запас" в выходной строке.

In [3]:
def calculate_suffix_len(string_len: int) -> int:
    "Расчет длины суффикса в зависимости от количества секций"
    sections = rounder_upper(string_len/140) # получаем примерное количество секций, которое будет в результате работы нашего ПО
    if sections <= 9: # 9 секций дают следующий формат вывода: N/N - а это три символа. Именно такое количество мы и резервируем
        return 3
    elif sections > 9 and sections <= 99: # от 10 до 99 секций дают следующий формат вывода: NN/NN - а это пять символов
        return 5
    elif sections > 99 and sections <= 999: # от 100 до 999 секций дают следующий формат вывода: NNN/NNN - а это семь символов
        return 7
    else: # по ТЗ количество секций не может быть больше 9999, поэтому ограничимся 9 резервными символами - NNNN/NNNN
        return 9

#### Демонстрация работы

In [4]:
k = 1
print('string len -> number of sections -> suffix len')
while k !=10:
    ran = random.randint(30, 50000)
    print(f'{ran} -> {rounder_upper(ran/140)} -> {calculate_suffix_len(ran)}')
    k+=1

string len -> number of sections -> suffix len
665 -> 5 -> 3
42031 -> 301 -> 7
39131 -> 280 -> 7
6013 -> 43 -> 5
4017 -> 29 -> 5
48938 -> 350 -> 7
42225 -> 302 -> 7
19216 -> 138 -> 7
15155 -> 109 -> 7


### line_separator - основная функция, выполняющая разделение строки на несколько секций.

ТЗ:

1. Функция принимает на вход текст, написанный латиницей, где все слова
разделены только 1 пробелом. Текст содержит в себе только латинские буквы и
пробелы (без знаков препинания).   
2. Задача функции разбить текст на СМСки размером не больше 140 символов и
вернуть в результате массив получившихся строк.   
3. Так как СМСки платные важно разбить текст на минимальное количество
фрагментов.   
4. Слова нельзя разбивать посередине, то есть текст нужно разбить строго по
пробелам.   
5. Если весь текст не помещается в один фрагмент то каждый фрагмент должен
заканчиваться суффиксом ' k/n', где k - порядковый номер фрагмента, n -
количество фрагментов, на которые разбит текст. k <= n <= 9999.   
6. Суффикс учитывается при подсчете длины СМСки, то есть длина фрагмента
вместе с суффиксом должна быть меньше или равна 140 символам.   
7. Задача решаема, то есть в тексте не может быть настолько длинных слов, чтобы
одно слово с суффиксом не уместилось в одном СМС.   

In [5]:
def line_separator(current_line: str) -> list:
    "Разделение строки на несколько блоков, содержащих максимум по 140 символов с добавлением суффикса"
    if len(current_line) <= 140:
        return [current_line]
    else: 
        total_len = 140 - calculate_suffix_len(len(current_line)) # получаем длину суффикса - именно такое количество символов должны мы зарезервировать из 140 символов
        sections_num = rounder_upper(len(current_line)/(total_len)) # считаем количество секций, которое будет в выходном сообщении
        split_line = current_line.split() # разделяем исходную строку по пробелу
        counter = 0 # добавляем счетчик и несколько пустых списков, для дальнейшей работы
        string = []
        total_string = []
        for element in split_line: # проходимся по всем элементам списка и проверяем их длину
            # если общая длина строки выходит за допустимую - объединяем получившуюся строку и продолжаем итерацию, очистив счетчик и временный список
            if counter+len(element) >= total_len: 
                counter = len(element)
                total_string.append(' '.join(string))
                string = [element]
            # иначе - продолжаем итерацию, до тех пор пока длина получившейся строки не достигает нужной длины
            else:
                counter+=len(element)
                string.append(element)
        total_string.append(' '.join(string)) # прибавляем в список последний получившийся элемент

        # теперь работаем с суффиксами - добавляем их к концу каждой строки
        counter = 1
        for i in range(0, len(total_string)):
            total_string[i] = total_string[i]+f' {counter}/{len(total_string)}'
            counter+=1
        return total_string

### Тесты

### Тест №1. Строка, содержащая до 140 символов.

In [6]:
line_separator(lorem.get_word(count=8))

['eiusmod laborum sit ullamco veniam aliquip magna labore']

#### Замеры времени выполнения по заготовленной строке
> Для объективной оценки времени выполнения, воспользуемся заготовленной строкой

In [7]:
%%timeit
line_separator('Lorem ipsum dolor sit amet consectetur adipiscing elit')

82.5 ns ± 0.731 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


> Обратите внимание, в данном тесте используется подключаемая библиотека с генератором текста, что может замедлить выполнение программы

In [8]:
%%timeit
line_separator(lorem.get_word(count=8))

113 µs ± 569 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Тест №2. Строка, содержащая свыше 140 символов.

> Обратите внимание, в данном тесте используется подключаемая библиотека с генератором текста, что может замедлить выполнение программы

In [9]:
line_separator(lorem.get_word(count=random.randint(100, 120)))

['incididunt ullamco ad eiusmod duis laborum ut tempor do sed officia et cillum ullamco aute incididunt ad pariatur amet reprehenderit ut nulla lorem culpa 1/5',
 'voluptate deserunt aliquip tempor duis nostrud irure consectetur elit fugiat nisi pariatur nulla non sint sit sint ex enim excepteur incididunt culpa esse aute 2/5',
 'proident labore est duis veniam sunt est officia et magna excepteur ut eiusmod ut ut et in officia lorem lorem culpa in minim nulla duis in labore anim dolor dolor 3/5',
 'proident eu sint incididunt exercitation nisi excepteur sed ipsum nulla duis aute esse nulla minim reprehenderit sed consectetur nulla incididunt pariatur 4/5',
 'ullamco ullamco 5/5']

#### Замер с использованием генератора слов

In [10]:
%%timeit
line_separator(lorem.get_word(count=random.randint(1000, 1200)))
# line_separator(sample_line)

15.9 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
sample_line = 'Lorem ipsum dolor sit amet consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua Faucibus turpis in eu mi bibendum neque egestas Risus pretium quam vulputate dignissim suspendisse in est ante Non curabitur gravida arcu ac tortor dignissim convallis aenean Lacus suspendisse faucibus interdum posuere lorem ipsum dolor sit Morbi tincidunt augue interdum velit euismod in pellentesque Quis hendrerit dolor magna eget est Tempus iaculis urna id volutpat lacus laoreet Tortor consequat id porta nibh venenatis Etiam tempor orci eu lobortis elementum nibh tellus molestie Tortor condimentum lacinia quis vel Placerat duis ultricies lacus sed turpis tincidunt Adipiscing enim eu turpis egestas pretium aenean Donec massa sapien faucibus et molestie ac feugiat sed lectus Dictum at tempor commodo ullamcorper a lacus vestibulum sed arcu Odio facilisis mauris sit amet massa vitae tortor condimentum lacinia Quis lectus nulla at volutpat diam ut venenatis tellus in Tellus cras adipiscing enim eu turpis egestas pretium aenean Nulla facilisi etiam dignissim diam quis enim lobortis Orci a scelerisque purus semper eget Nunc eget lorem dolor sed viverra ipsum nunc aliquet Scelerisque viverra mauris in aliquam sem fringilla ut Sit amet tellus cras adipiscing enim Volutpat diam ut venenatis tellus in metus vulputate eu scelerisque Arcu dui vivamus arcu felis bibendum Sagittis orci a scelerisque purus semper eget duis at tellus A diam maecenas sed enim ut sem Scelerisque purus semper eget duis Pharetra vel turpis nunc eget lorem dolor sed Non tellus orci ac auctor augue mauris augue neque gravida Proin sagittis nisl rhoncus mattis rhoncus Ipsum dolor sit amet consectetur adipiscing elit ut aliquam Lacus sed turpis tincidunt id Vel pharetra vel turpis nunc eget Massa eget egestas purus viverra accumsan Vestibulum lectus mauris ultrices eros in cursus Eget gravida cum sociis natoque penatibus et Lacus suspendisse faucibus interdum posuere lorem ipsum dolor sit amet Nec feugiat nisl pretium fusce id velit Semper quis lectus nulla at volutpat Vitae proin sagittis nisl rhoncus Mattis vulputate enim nulla aliquet porttitor lacus luctus accumsan tortor Arcu non odio euismod lacinia at Proin sed libero enim sed faucibus Lacus suspendisse faucibus interdum posuere lorem ipsum Sagittis orci a scelerisque purus semper eget duis at Suscipit adipiscing bibendum est ultricies integer quis auctor Egestas pretium aenean pharetra magna ac placerat Pulvinar sapien et ligula ullamcorper Nulla facilisi morbi tempus iaculis urna id volutpat lacus laoreet Placerat vestibulum lectus mauris ultrices eros in cursus Tristique et egestas quis ipsum suspendisse ultrices Diam phasellus vestibulum lorem sed risus ultricies Id volutpat lacus laoreet non curabitur gravida arcu Sed blandit libero volutpat sed cras ornare arcu dui vivamus In nibh mauris cursus mattis molestie a iaculis Dolor purus non enim praesent elementum facilisis Maecenas volutpat blandit aliquam etiam erat velit Odio euismod lacinia at quis risus Turpis egestas pretium aenean pharetra Integer enim neque volutpat ac tincidunt vitae Rhoncus mattis rhoncus urna neque viverra Nunc sed velit dignissim sodales ut eu sem Iaculis urna id volutpat lacus laoreet non curabitur gravida At quis risus sed vulputate odio ut enim Consequat nisl vel pretium lectus quam id leo in vitae Egestas fringilla phasellus faucibus scelerisque In iaculis nunc sed augue lacus viverra Est sit amet facilisis magna etiam tempor orci eu lobortis Quam id leo in vitae turpis Ipsum dolor sit amet consectetur adipiscing elit pellentesque habitant morbi'

In [13]:
%%timeit
line_separator(sample_line)

69.6 µs ± 930 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
line_separator(sample_line)

['Lorem ipsum dolor sit amet consectetur adipiscing elit sed do eiusmod tempor incididunt ut labore et dolore magna aliqua Faucibus turpis in eu mi bibendum 1/24',
 'neque egestas Risus pretium quam vulputate dignissim suspendisse in est ante Non curabitur gravida arcu ac tortor dignissim convallis aenean Lacus 2/24',
 'suspendisse faucibus interdum posuere lorem ipsum dolor sit Morbi tincidunt augue interdum velit euismod in pellentesque Quis hendrerit dolor magna eget 3/24',
 'est Tempus iaculis urna id volutpat lacus laoreet Tortor consequat id porta nibh venenatis Etiam tempor orci eu lobortis elementum nibh tellus molestie 4/24',
 'Tortor condimentum lacinia quis vel Placerat duis ultricies lacus sed turpis tincidunt Adipiscing enim eu turpis egestas pretium aenean Donec massa sapien 5/24',
 'faucibus et molestie ac feugiat sed lectus Dictum at tempor commodo ullamcorper a lacus vestibulum sed arcu Odio facilisis mauris sit amet massa vitae tortor 6/24',
 'condimentum lacinia Quis